In [ ]:
# Install all of the things
!pip3 install numpy
!pip3 install pandas
#!pip install statsmodels
!pip3 install sklearn
!pip3 install scipy
#!pip install nltk
!pip3 install matplotlib
!pip3 install pandasql

In [1]:
import pandas as pd
import numpy as np
#import statsmodels
import sklearn
import scipy
import matplotlib
import datetime
from pytz import timezone
import pandasql as pdsql

In [2]:
#Import all data
printer_data = pd.read_csv('http://ocf.io/shichenh/ocf_datathon_ds/printing.csv')
session_data = pd.read_csv('https://www.ocf.berkeley.edu/~shichenh/ocf_datathon_ds/sessions.csv')
staff_data = pd.read_csv('https://www.ocf.berkeley.edu/~shichenh/ocf_datathon_ds/s_sessions.csv')

In [3]:
#Trim the printer data to have no entries past the time 2017-11-09 20:00:00 UTC (Noon pacific)
#Also, no entries before 9AM Pacific 2016-03-16
printer_data = printer_data[39:194767]

#Trim the session data to have no entries past the time 2017-11-09 20:00:00 UTC (noon pacific)
#Also, no entries before 16:00:00 UTC
session_data = session_data[156305:302446]


In [4]:
#Convert dates and times into a datetime object
session_data['start'] = pd.to_datetime(session_data['start'], format='%Y-%m-%dT%H:%M:%SZ')
session_data['end'] = pd.to_datetime(session_data['end'], format='%Y-%m-%dT%H:%M:%SZ')
printer_data['time'] = pd.to_datetime(printer_data['time'], format='%Y-%m-%d %H:%M:%S')

printer_data['strtime'] = printer_data['time'].dt.tz_localize('US/Pacific').dt.tz_convert('utc')
printer_data['strtime'].apply(lambda x: x.strftime("%Y:%m:%d:%H:%M:%S"))

session_data['start'] = session_data['start'].dt.tz_localize('utc')
session_data['start'].apply(lambda x: x.strftime("%Y:%m:%d:%H:%M:%S"))

session_data['end'] = session_data['end'].dt.tz_localize('utc')
session_data['end'].apply(lambda x: x.strftime("%Y:%m:%d:%H:%M:%S"))

print("No Output Desired")

No Output Desired


In [5]:
session_data.user.apply(str)
printer_data.user.apply(str)

printer_data['strtime'] = printer_data['strtime'].astype(str)
session_data['start'] = session_data['start'].astype(str)
session_data['end'] = session_data['end'].astype(str)

print("No Output Desired")


No Output Desired


In [23]:
#Attempt to merge using SQL
query = """SELECT * FROM session_data a INNER JOIN printer_data b 
            ON a.user = b.user WHERE a.start < b.strtime AND a.end > b.strtime"""

pysql = lambda q: pdsql.sqldf(q, globals())

merged_table = pysql(query)

In [24]:
merged_table = merged_table.drop(columns=['host', 'last_update', 
                                          'Unnamed: 0', 'queue', 'date', 
                                          'weekday', 'strtime'])

In [25]:
merged_table.columns = ['id', 'user', 'start', 
                        'end', 'duration', 'time', 
                        'pages', 'user2']
merged_table = merged_table.drop(columns=['user2'])

In [26]:
#Sum print jobs in the same session
query = """ SELECT *, SUM(pages) FROM merged_table t GROUP BY start
"""

pysql = lambda q: pdsql.sqldf(q, globals())

printer_usage = pysql(query)

In [31]:
printer_usage.rename(columns={'SUM(pages)' : 'total_printed'}, inplace=True)
printer_usage.drop(columns=['pages'])
print('No Output Desired')

No Output Desired


In [19]:
print_sessions = printer_usage.query('pages >= 1')
print_sessions.describe()

id      duration     Unnamed: 0         pages    SUM(pages)
count   97295.000000  97295.000000   97295.000000  97295.000000  97295.000000
mean   224968.252963     15.471745   93179.521281      3.657177      6.059983
std     42160.263072     49.194325   56742.281266      3.369518      6.010579
min    156307.000000      0.100000      40.000000      1.000000      1.000000
25%    189173.000000      2.816667   44734.500000      1.000000      2.000000
50%    220572.000000      5.350000   85861.000000      2.000000      5.000000
75%    261277.000000     11.866667  144104.500000      5.000000      8.000000
max    302441.000000   3504.983333  194766.000000    254.000000    600.000000

In [20]:
noprint_sessions = printer_usage.query('pages <= 0')
noprint_sessions.describe()

id    duration     Unnamed: 0  pages  SUM(pages)
count      34.000000   34.000000      34.000000   34.0   34.000000
mean   158923.823529   18.607353    3661.735294    0.0    5.000000
std     13548.009847   32.631570   18390.781374    0.0    3.684529
min    156314.000000    1.433333      60.000000    0.0    0.000000
25%    156421.750000    5.187500     226.500000    0.0    0.250000
50%    156530.500000    8.558333     412.000000    0.0    6.000000
75%    156670.750000   14.454167     653.000000    0.0    8.000000
max    235559.000000  171.266667  107682.000000    0.0   14.000000